In [13]:
%env WANDB_MODE="dryrun"

env: WANDB_MODE=&quot;dryrun&quot;


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from src.models.base import MLP

In [18]:
import yaml
from src.experiments import Fusion_MVAE_Experiment, MVAE_Tester
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl

In [4]:
config = "configs/mnist_svhn/fusion.yaml"

with open(config, "r") as file:
    try:
        hparams = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

In [10]:
exp = Fusion_MVAE_Experiment(hparams)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [16]:
# Init logger
project_name = "vae-expts"
wandb_logger = WandbLogger(
    name=hparams["name"], project=project_name, id=hparams["name"],  # For resuming
)
wandb_logger.log_hyperparams(hparams)

&lt;IPython.core.display.HTML object&gt;

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.10.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [19]:
trainer = pl.Trainer(
        fast_dev_run=True,
        deterministic=True,
        benchmark=True,
        gpus=1,
        logger=wandb_logger,
        weights_summary="top",
        max_epochs=1,
        val_check_interval=0.25,
        # limit_val_batches=0.,
)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [20]:
trainer.fit(exp)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.10.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

  | Name  | Type          | Params
----------------------------------------
0 | model | MultimodalVAE | 1 M   


Saving latest checkpoint..



1